In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data

In [7]:
class Word2VecModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, batch_size, bag_size, skip_gram=True):
        super().__init__()
        self.vocab_size    = vocab_size
        self.embedding_dim = embedding_dim
        self.batch_size    = batch_size
        self.bag_size      = bag_size
        self.skip_gram     = skip_gram
        
        self.embedding_layer = nn.Embedding(self.vocab_size, self.embedding_dim)
        self.linear_layer    = nn.Linear(self.embedding_dim, self.vocab_size)
        
    def forward(self, inputs):
        #inputs = torch.tensor(inputs)
        if self.skip_gram:
            #inputs size = [batch] (value is index)
            embedding_vectors = self.embedding_layer(inputs)       # embedding_vectors size = [batch, embedding_dim]
        else:
            #inputs size = [batch, bag_size]
            embedding_vectors = []
            inputs = inputs.view(-1)                               # inputs size = [batch * bag_size]
            embedding_vectors = self.embedding_layer(inputs)       # embedding_vectors size = [batch * bag_size, embedding_dim]
            embedding_vectors = embedding_vectors.view(self.batch_size, -1, self.embedding_dim) #embedding_vectors size = [batch, bag_size, embedding_dim]
            embedding_vectors = torch.div(torch.sum(embedding_vectors, 1), self.bag_size) # embedding_vectors size = [batch, embedding_dim]
        vocab_vectors = self.linear_layer(embedding_vectors)       #vocab_vectors size = [batch, vocab_size]
        preds = F.softmax(vocab_vectors, dim=1)
        return preds
    
    def inference(self, inputs):
        #inputs = torch.tensor(inputs) #input size = [batch, seq_len]
        inputs = inputs.view(-1) # inputs size = [batch * seq_len]
        embedding_vectors = self.embedding_layer(inputs) # embedding_vectors size = [batch * seq_len, embedding_dim]
        embedding_vectors = embedding_vectors.view(self.batch_size, -1, self.embedding_dim) # preds size = [batch, seq_len, embedding_dim]
        return embedding_vectors